In [1]:
from PIL import Image
import numpy as np
import math
import random
from math import cos, sin

# Лабротароные работы №3-4

In [2]:
#назначаем размеры изображения
H = 700
W = 1000

In [3]:
#создаем изображение
image = np.zeros((H,W,3), dtype=np.uint8)

In [4]:
#черное изображение
for i in range(H):
    for j in range(W):
        image[i,j,0], image[i,j,1], image[i,j,2] = 0, 0, 0

In [5]:
file = open('Fox.obj', 'r')
v = []
#считываем все вершины
for i in file:
    if i.split()[0] == 'v':
        v.append((float(i.split()[2]), float(i.split()[3]), float(i.split()[1])))

f = []
file = open('Fox.obj', 'r')
#считываем все полигоны
for i in file:
    if i.split()[0] == 'f':
        s = i.split()
        f.append((s[1].split('/')[0], s[2].split('/')[0], s[3].split('/')[0]))



In [6]:
#реализуем матрицу поворота
a = 1 #угол, который стоит трогать, если хочешь покрутить модельку
b = 0 #угол, который можно трогать, но не стоит
c = 0 #угол, который бесполезно трогать
R1 = np.array([[1,0,0],[0,cos(a),sin(a)],[0,-sin(a),cos(a)]])
R2 = np.array([[cos(b),0,sin(b)],[0,1,0],[-sin(b),0,cos(b)]])
R3 = np.array([[cos(c),sin(c),0],[-sin(c),cos(c),0],[0,0,1]])
R = np.dot(R1, R2, R3) #матрица поворота
vvv = []
#умножаем матрицу из исходных координат вершин, на матрицу поворота
for i in v:
    A = np.dot(R, np.array([[i[0]],[i[1]],[i[2]]]))
    x = float(A[0])
    y = float(A[1])
    z = float(A[2])
    vvv.append([x, y, z])

In [7]:
#Пункт 16 из методички
vv = []
vvn=[]
for i in vvv:
    #проективное преобразование
    #1000 - масштаб, (500, 500) - центр изображения
    B = np.array([[1000, 0, 500],[0, 1000, 500],[0, 0, 1]])
    C = np.array([[-i[0]],[i[1]],[i[2]+250]]) #250 - насколько отдаляем модельку от экрана
    A = np.dot(B,C)
    
    
    A = A / (i[2]+250) #делим на координату z

    
    x = float(A[0])
    y = float(A[1])
    z = float(A[2])
    vvn.append([x,y,z])
    
    
    x = float(A[0])
    y = float(A[1])
    z = float(i[2]+250) #костыль для того чтобы работал свет

    vv.append([x,y,z])

In [8]:
#освещение модельки, полностью взято из прошлой лабы. Пункты 8-15 методички
buffer = []
for k in range(H):
    buffer.append([500 for i in range(W)])
        
for i in f: 


    x0 = float(vvn[int(i[0])-1][0])
    x1 = float(vvn[int(i[1])-1][0])
    x2 = float(vvn[int(i[2])-1][0])
    y0 = float(vvn[int(i[0])-1][1])
    y1 = float(vvn[int(i[1])-1][1])
    y2 = float(vvn[int(i[2])-1][1])
    
    z0 = float(vvn[int(i[0])-1][2])
    z1 = float(vvn[int(i[1])-1][2])
    z2 = float(vvn[int(i[2])-1][2])
    
    
    
 
    n = ((y1-y0)*(z1-z2)-(y1-y2)*(z1-z0), (z1-z0)*(x1-x2)-(x1-x0)*(z1-z2), (x1-x0)*(y1-y2)-(y1-y0)*(x1-x2))
    l = (-0.5, 0.5, -0.5)
    
    cos_nl = (n[0]*l[0]+n[1]*l[1]+n[2]*l[2])/(((n[0]**2+n[1]**2+n[2]**2)**0.5) * ((l[0]**2+l[1]**2+l[2]**2)**0.5))

    
    
    
    n0, n1, n2 = [], [], []
    
    for j in f:
        xx0 = float(vv[int(j[0])-1][0])
        xx1 = float(vv[int(j[1])-1][0])
        xx2 = float(vv[int(j[2])-1][0])
        yy0 = float(vv[int(j[0])-1][1])
        yy1 = float(vv[int(j[1])-1][1])
        yy2 = float(vv[int(j[2])-1][1])

        zz0 = float(vv[int(j[0])-1][2])
        zz1 = float(vv[int(j[1])-1][2])
        zz2 = float(vv[int(j[2])-1][2])
        
        
        if j[0] == i[0] or j[1] == i[0] or j[2] == i[0]:
            n = ((yy1-yy0)*(zz1-zz2)-(yy1-yy2)*(zz1-zz0), (zz1-zz0)*(xx1-xx2)-(xx1-xx0)*(zz1-zz2), (xx1-xx0)*(yy1-yy2)-(yy1-yy0)*(xx1-xx2))
            n0.append(n)

            
        if j[1] == i[1] or j[0] == i[1] or j[2] == i[1]:
            n = ((yy1-yy0)*(zz1-zz2)-(yy1-yy2)*(zz1-zz0), (zz1-zz0)*(xx1-xx2)-(xx1-xx0)*(zz1-zz2), (xx1-xx0)*(yy1-yy2)-(yy1-yy0)*(xx1-xx2))
            n1.append(n)
            
            
        if j[2] == i[2] or j[0] == i[2] or j[1] == i[2]:
            n = ((yy1-yy0)*(zz1-zz2)-(yy1-yy2)*(zz1-zz0), (zz1-zz0)*(xx1-xx2)-(xx1-xx0)*(zz1-zz2), (xx1-xx0)*(yy1-yy2)-(yy1-yy0)*(xx1-xx2))
            n2.append(n)
            
            
    nn0 = [0,0,0]
    nn1 = [0,0,0]
    nn2 = [0,0,0]
    for k in n0:
        nn0[0] -= k[0]
        nn0[1] -= k[1]
        nn0[2] -= k[2]
    nn0[0] /= len(n0)
    nn0[1] /= len(n0)
    nn0[2] /= len(n0)
    for k in n1:
        nn1[0] -= k[0]
        nn1[1] -= k[1]
        nn1[2] -= k[2]
    nn1[0] /= len(n1)
    nn1[1] /= len(n1)
    nn1[2] /= len(n1)
    for k in n2:
        nn2[0] -= k[0]
        nn2[1] -= k[1]
        nn2[2] -= k[2]
    nn2[0] /= len(n2)
    nn2[1] /= len(n2)
    nn2[2] /= len(n2)
    n0.clear()
    n1.clear()
    n2.clear()
    
    I0 = abs((nn0[0]*l[0]+nn0[1]*l[1]+nn0[2]*l[2]) / ((nn0[0]**2+nn0[1]**2+nn0[2]**2)**0.5 + (l[0]**2+l[1]**2+l[2]**2)**0.5))
    I1 = abs((nn1[0]*l[0]+nn1[1]*l[1]+nn1[2]*l[2]) / ((nn1[0]**2+nn1[1]**2+nn1[2]**2)**0.5 + (l[0]**2+l[1]**2+l[2]**2)**0.5))
    I2 = abs((nn2[0]*l[0]+nn2[1]*l[1]+nn2[2]*l[2]) / ((nn2[0]**2+nn2[1]**2+nn2[2]**2)**0.5 + (l[0]**2+l[1]**2+l[2]**2)**0.5))

    
    
    
    if cos_nl <= 0:
        continue
        
        
    for x in range(int(min(x0,x1,x2)), int(max(x0,x1,x2))+1):
        for y in range(int(min(y0,y1,y2)), int(max(y0,y1,y2))+1):
            
            xx = float(x)
            xx0 = float(vv[int(i[0])-1][0])
            xx1 = float(vv[int(i[1])-1][0])
            xx2 = float(vv[int(i[2])-1][0])
            yy0 = float(vv[int(i[0])-1][1])
            yy1 = float(vv[int(i[1])-1][1])
            yy2 = float(vv[int(i[2])-1][1])
            yy = float(y)
            
            lambda0 = ((xx1 - xx2)*(yy - yy2) - (yy1 - yy2)*(xx - xx2)) / ((xx1 - xx2)*(yy0 - yy2) - (yy1 - yy2)*(xx0 - xx2))
            if lambda0 <= 0:
                continue
            lambda1 = ((xx2 - xx0)*(yy - yy0) - (yy2 - yy0)*(xx - xx0)) / ((xx2 - xx0)*(yy1 - yy0) - (yy2 - yy0)*(xx1 - xx0))
            if lambda1 <= 0:
                continue
            lambda2 = ((xx0 - xx1)*(yy - yy1) - (yy0 - yy1)*(xx - xx1)) / ((xx0 - xx1)*(yy2 - yy1) - (yy0 - yy1)*(xx2 - xx1))
            if lambda2 <= 0:
                continue
                
            I = abs(I0*lambda0 + I1*lambda1 + I2*lambda2)
            #print(I)
            #print(I, lambda0+lambda1+lambda2)
            col1 = 255*I
            if col1>255:
                col1 = 255
            col2 = 171*I
            col3 = 0*I
            if vv[int(i[0])-1][2]*lambda0 + vv[int(i[1])-1][2]*lambda1 + vv[int(i[2])-1][2]*lambda2 < buffer[x][y]:
                image[x,y,0], image[x,y,1], image[x,y,2] = col1, col2, col3
                buffer[x][y] = vv[int(i[0])-1][2]*lambda0 + vv[int(i[1])-1][2]*lambda1 + vv[int(i[2])-1][2]*lambda2

In [9]:
#сохранение изображения
image = Image.fromarray(image, 'RGB')
image.save("image.jpg")